In [ ]:
#How to find, load and process snRNA-seq data

In [ ]:
import wget
import pandas as pd
import numpy as np
import scanpy as sc
import anndata

In [ ]:
#Gene network analysis is a method designed to identify sub-networks (modules) of correlated genes, which are likely to be co-expressed.
#This can be helpful in identification of sub-networks (modules) of genes that contribute to disease.
#In this example, we will cover how to create a pairwise correlation matrix of genes, as well as how to associate them with disease.

In [ ]:
#First we will cover how to find, load and process the snRNA-seq data

In [ ]:
#Acquire snRNA-seq data from cellxgene portal: https://cellxgene.cziscience.com/collections/180bff9c-c8a5-4539-b13b-ddbc00d643e6
#Chosen microglia cell type to focus on from this paper: Molecular characterization of selectively vulnerable neurons in Alzheimer's Disease
#https://www.nature.com/articles/s41593-020-00764-7

In [ ]:
#For this tutorial, we will be using an open access freely available dataset that has been generated from microglia of the entorhinal cortex within the brain.
#This dataset is available from the cellxgene portal, accessible here: https://cellxgene.cziscience.com/collections/180bff9c-c8a5-4539-b13b-ddbc00d643e6 entitled "Molecular characterization of selectively vulnerable neurons in Alzheimer’s Disease: EC microglia".
#SnRNA-seq was performed for Controls and donors with Alzheimer's Disease.
#This dataset was chosen due to its small size and compatability with the purpose of the pipeline.
#This data will be available in the data/test/ directory.
#The generated dataset is stored in h5ad format.
#By the end of this section, we will have loaded and explored the dataset.


In [ ]:
#Start by downloading the dataset from the original portal.
# URL of the dataset
url = "https://datasets.cellxgene.cziscience.com/1f0cd8ed-94c6-440c-bd5b-bad55e2666b1.h5ad"

# Destination path where the dataset will be saved
destination_path = "/shared/as8020/recode/mic_leng21.h5ad"

# Download the dataset
wget.download(url, destination_path)

#Alternatively, the dataset can be found in the dataset/test/ directory saved as mic_leng21.h5ad.

In [ ]:
#Load in the test dataset
mic = sc.read('dataset/mic_leng21.h5ad')

In [ ]:
#inspect the loaded data
mic

In [ ]:
#Check if the gene names are in the correct format of gene symbols and not Ensembl IDs which are also common.
mic.var

In [ ]:
#As can be seen from the gene features dataframe, they have currently used the Ensembl gene naming system.
#However, this isn't helpful for our analyses as they are not intuitively easy to interpret, instead you would need to research each Ensembl ID to identify that particular gene's name and function.
#From the second column feature_name, it appears that the original authors have converted the Ensembl IDs to gene symbol names.

#Let's go ahead and map the values in the feature_name column to the rownames of the dataframe:
# Set the "feature_name" column as the index (row names)
mic.var.set_index("feature_name", drop = False, inplace=True)

#It is important to note that not all Ensembl IDs map to Gene symbol names, as can be seen within rows 3 and 5 within the top of the dataframe.
#Therefore, since there is not a mapping for all Ensembl IDs, we shall remove these rows from the dataframe as they will be difficult to interpret in subsequent analyses.
# Filter rows where the index does not start with "ENSG" i.e. the Ensembl IDs.
# Define the condition for filtering genes
filter_genes = ~mic.var.index.str.startswith("ENSG")  # Exclude genes starting with "ENSG"
filter_genes

# Filter genes based on the condition
mic = mic[:, filter_genes]


In [ ]:
mic

In [ ]:
mic.var

In [ ]:
#As can be seen, the number of genes have now reduced as any rows with Ensembl IDs have been removed.

In [ ]:
#Also calculate the highly variable genes.

In [ ]:
#Calculating highly variable genes on gene expression data that has not been log-transformed or normalized appropriately can lead to issues, including the presence of infinity values.
#Log transformation is a common preprocessing step for scRNA-seq data, especially when dealing with count data, to stabilize the variance and make the data more amenable to downstream analysis. 
#It helps to mitigate the impact of high expression values and reduce the influence of technical noise.

In [ ]:
# Log normalize the gene expression data
sc.pp.log1p(mic)

In [ ]:
# Calculate highly variable genes
sc.pp.highly_variable_genes(mic, n_top_genes = 1000)

In [ ]:
mic

In [ ]:
#Lets save the filtered object
mic.write_h5ad('dataset/mic_leng21_filtered.h5ad')

In [ ]:
#We will now explore the associated metadata 

In [ ]:
mic.obs.columns

In [ ]:
#As can be seen, this dataset contains 5572 cells and 32743 genes.
#It also has relevant metadata in the obs section, such as BraakStage. 
#The metadata may need to be encoded into the correct format for subsequent analyses, so let's have a look at the current format.

In [ ]:
mic.obs

In [ ]:
#Lets create a separate dataframe with the metadata information as this will be needed for the correlation analysis.
#Currently we want to create a copy of the metadata so as not to alter the original adata object.
metadata = mic.obs.copy()
metadata

In [ ]:
#There are many columns that are not needed.
#Let's remove uninteresting columns
columns_to_remove = ['SampleID', 'SampleBatch',
       'initialClusterAssignments',
       'subclusterAssignment', 'tissue_ontology_term_id',
       'cell_type_ontology_term_id', 'assay_ontology_term_id',
       'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id',
       'development_stage_ontology_term_id', 'sex_ontology_term_id',
       'is_primary_data', 'organism_ontology_term_id', 'suspension_type',
       'tissue_type', 'cell_type', 'assay', 'organism',
       'tissue', 'self_reported_ethnicity',
       'observation_joinid' ]

In [ ]:
metadata.drop(columns=columns_to_remove, inplace = True) #Set inplace=True to modify the DataFrame in place. If you set inplace=False or omit it, the drop() method will return a new DataFrame with the specified columns removed, leaving the original DataFrame unchanged.

In [ ]:
metadata

In [ ]:
mic

In [ ]:
#From investigating the metadata dataframe, BraakStage, nUMI, nGene and seurat.subclusters are all numerical, whilst disease, sex and development_stage are all character strings.
#The columns with character strings will need to be reformatted appropriately so that they can be correlated against.
#Lets first identify the unique labels within each column

In [ ]:
metadata['sex'].unique()

In [ ]:
#Looks like there are only male participants. Since there are only male differences, this column can also be removed, since we will not be able to investigate sex differences.

In [ ]:
column_to_remove = 'sex'
metadata.drop(columns=column_to_remove, inplace = True)

In [ ]:
metadata

In [ ]:
#Now let's have a look at the disease variable
metadata['disease'].unique()

In [ ]:
#The disease column can be encoded into a binary variable:
metadata['AD'] = metadata['disease'].apply(lambda x: 1 if x == "Alzheimer disease" else 0)
metadata['Normal'] = metadata['disease'].apply(lambda x: 1 if x == "normal" else 0)

In [ ]:
metadata

In [ ]:
#Now lets sort out the development_stage column

In [ ]:
metadata['development_stage'].unique()

In [ ]:
#There appear to be 8 categories. Lets numerically encode them
# Recode development_stage
development_stage_mapping = {
    '50-year-old human stage': 50,
    '60-year-old human stage': 60,
    '71-year-old human stage': 71,
    '72-year-old human stage': 72,
    '77-year-old human stage': 77,
    '80 year-old and over human stage': 80,
    '82-year-old human stage': 82,
    '87-year-old human stage': 87
}
metadata['development_stage'] = metadata['development_stage'].map(development_stage_mapping)

In [ ]:
metadata

In [ ]:
#Drop the disease column as it is no longer necessary
# Drop unnecessary columns
metadata = metadata.drop(['disease'], axis=1)
metadata

In [ ]:
#Save the metadata dataframe
metadata.to_csv('data/mic_metadata.csv', index = True)

In [ ]:
metadata = pd.read_csv('data/mic_metadata.csv', index_col = 0)

In [ ]:
#Due to the nature of single-cell data, we naturally have many cells from the same donor.
#However, we cannot simply correlate the gene expression data in its current form. this would lead to within and outwith donor correlations.
#Therefore, since we are working with single-cell data, this must first be pseudobulked in order to continue with the analysis.
#This is important as it not only speeds up the computation, but most importantly negates the effects of within sample correlation.
#Also, pseudobulking can help to mitigate the issues commonly found in single-cell data, such as drop outs and high zero value counts.

In [ ]:
#First we shall sort out the metadata dataframe so that it only contains one row per donor since the data will be aggregated.

In [ ]:
# Convert row names to a column named 'cell_id'
metadata['cell_id'] = metadata.index

In [ ]:
# Group by 'donor_id' and select the first row of each group
rows = metadata.groupby('donor_id').first().reset_index()

In [ ]:
rows

In [ ]:
# Extract row indices corresponding to the first cell from each donor
row_list = []
for i, row in rows.iterrows():
    row_idx = metadata.index.get_loc(row['cell_id'])
    row_list.append(row_idx)

In [ ]:
row_list

In [ ]:
# Select the columns from the DataFrame
metadata2 = metadata.iloc[row_list, :].copy()

In [ ]:
metadata2

In [ ]:
metadata2.set_index('donor_id', inplace = True, drop = False)

In [ ]:
metadata2

In [ ]:
#Remove the cell_id column
metadata2.drop(columns = 'cell_id', inplace = True)

In [ ]:
metadata2

In [ ]:
#Save the metadata
metadata2.to_csv('data//mic_metadata_pseudobulk.csv', index = True)

In [ ]:
#The metadata dataframe for the pseudobulk is now complete

In [ ]:
#Lets proceed to aggregate the gene expression data.
#This involves summing the gene expression data for each gene of each donor.

In [ ]:
#First the gene expression matrix will need to be extracted from our mic adata object

In [ ]:
#since we are working with single-cell data which will be stored as a sparse matrix, this must be coerced into a dense matrix, so that it can be converted to a dataframe.

In [ ]:
# Convert the sparse matrix to a dense matrix
dense_matrix = mic.X.todense()

In [ ]:
datExpr = pd.DataFrame(dense_matrix, index=mic.obs_names, columns=mic.var_names)

In [ ]:
datExpr

In [ ]:
#save datExpr
#Save the metadata dataframe
datExpr.to_csv('dataset/mic_datExpr_singlecell.csv', index = True)

In [ ]:
#Since highly variable genes capture the most informative genes, they will be used to filter the expression matrix further.
#This is also a way to reduce the dimensionality of the data, so that downstream analyses may be more computationally efficient.

In [ ]:
hvg = mic.var_names[mic.var['highly_variable']]
hvg

In [ ]:
datExpr = datExpr.loc[:,hvg]
datExpr

In [ ]:
#Add the donor_id column to the gene expression dataframe, so we know which cell came from which donor

In [ ]:
# Reset the index of 'datExpr' DataFrame to make the row names (cell names) a column
datExpr_donor = datExpr.reset_index()

In [ ]:
datExpr_donor

In [ ]:
# Merge 'datExpr_reset' with 'metadata' on the 'index' and 'cell_id' columns
datExpr_donor = pd.merge(datExpr_donor, metadata[['cell_id', 'donor_id']], left_on='index', right_on='cell_id', how='left')

In [ ]:
datExpr_donor

In [ ]:
# Set the cell names as the index again
datExpr_donor.set_index('index', inplace=True)


In [ ]:
datExpr_donor

In [ ]:
# Remove the 'cell_id' column if needed
datExpr_donor.drop(columns=['cell_id'], inplace=True)

In [ ]:
datExpr_donor

In [ ]:
#Save the expression matrix with donor_id
datExpr_donor.to_csv('dataset/mic_datExpr_donorid_singlecell.csv', index = True)

In [ ]:
#Now that we have our gene expression dataframe, it is now possible to aggregate the data for pseudobulking.

In [ ]:
# Aggregate expression by donor ID (summing the values)
pseudobulk_df = datExpr_donor.groupby('donor_id').sum()

In [ ]:
pseudobulk_df

In [ ]:
#Save the pseudobulk expression matrix with donor_id
pseudobulk_df.to_csv('dataset/mic_datExpr_pseudobulk.csv', index = True)

In [ ]:
#We now have the pseudobulked data and the corresponding metadata dataframe to start the correlation network analysis